# tflite infernece

In [4]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image

tflite_path = "tflite/model.tflite"
image_path = "65888.jpg"

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get input shape
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

# Function to preprocess image
def preprocess_image(image_path, input_shape, input_dtype):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((input_shape[1], input_shape[2]))  # Resize to match model input
    image = np.array(image, dtype=np.float32)

    if input_dtype == np.uint8:  # If the model expects uint8 input
        image = image.astype(np.uint8)
    else:  # Normalize if model expects float32 input
        image = image / 255.0

    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Load and preprocess the image
image_path = image_path  # Change this to your image path
image = preprocess_image(image_path, input_shape, input_dtype)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], image)

# Run inference
interpreter.invoke()

# Retrieve results
scores = interpreter.get_tensor(output_details[0]['index'])  # Bounding boxes
boxes = interpreter.get_tensor(output_details[1]['index'])  # Class IDs
number_obj = interpreter.get_tensor(output_details[2]['index'])  # Confidence scores
classes = interpreter.get_tensor(output_details[3]['index'])  # Confidence scores

In [5]:
classes

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

In [6]:
scores

array([[0.80803525, 0.16386388, 0.10178218, 0.0799556 , 0.03316701,
        0.02894149, 0.02824223, 0.0280828 , 0.02693336, 0.02679672]],
      dtype=float32)

# temp

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

2025-03-11 14:58:33.188231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 14:58:33.437964: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 14:58:34.526936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2025-03-11 14:58:34.527049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [10]:
# Load the model
# model_path = "inference_graph/saved_model"  # Update with your model path
# model_path = "inference_graph_resnet/saved_model"  # Update with your model path
# model_path = "inference_graph_mbv2_640/saved_model"  # Update with your model path
model_path = "tflite_mbv2_640/saved_model"  # Update with your model path
detect_fn = tf.saved_model.load(model_path)

In [11]:
category_index = {1:'good', 2:'bad'}

# Load and preprocess an image
image_path = "dataset/check/good2.jpeg"
image_np = cv2.imread(image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

# Convert image to tensor
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]  # Add batch dimension

# Run inference
detections = detect_fn(input_tensor)

# Extract results
num_detections = int(detections.pop('num_detections'))
detection_boxes = detections['detection_boxes'][0].numpy()
detection_classes = detections['detection_classes'][0].numpy().astype(np.int32)
detection_scores = detections['detection_scores'][0].numpy()

# Draw bounding boxes
for i in range(num_detections):
    if detection_scores[i] > 0.5:  # Confidence threshold
        h, w, _ = image_np.shape
        y_min, x_min, y_max, x_max = detection_boxes[i]
        x_min, x_max, y_min, y_max = int(x_min * w), int(x_max * w), int(y_min * h), int(y_max * h)
        cv2.rectangle(image_np, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        
        class_label = category_index.get(detection_classes[i], f"Class {detection_classes[i]}")
        label = f"{class_label}: {detection_scores[i]:.2f}"
        # text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2, 1)[0]
        text_x, text_y = x_min, y_min - 5
        cv2.putText(image_np, label, (text_x, text_y - 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)

fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(image_np)
ax.axis('off')  # Optional: Hide axes
plt.show()


TypeError: '_UserObject' object is not callable

In [3]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="tflite_mbv2_640/model.tflite")
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get input shape
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

# Function to preprocess image
def preprocess_image(image_path, input_shape, input_dtype):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((input_shape[1], input_shape[2]))  # Resize to match model input
    image = np.array(image, dtype=np.float32)

    if input_dtype == np.uint8:  # If the model expects uint8 input
        image = image.astype(np.uint8)
    else:  # Normalize if model expects float32 input
        image = image / 255.0

    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Load and preprocess the image
image_path = "../dataset/check/good1.jpeg"  # Change this to your image path
image = preprocess_image(image_path, input_shape, input_dtype)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], image)

# Run inference
interpreter.invoke()

# Retrieve results
scores = interpreter.get_tensor(output_details[0]['index'])  # Bounding boxes
boxes = interpreter.get_tensor(output_details[1]['index'])  # Class IDs
number_obj = interpreter.get_tensor(output_details[2]['index'])  # Confidence scores
classes = interpreter.get_tensor(output_details[3]['index'])  # Confidence scores

In [4]:
boxes

array([[[ 0.37659684,  0.10950193,  0.744045  ,  0.85436153],
        [ 0.02218023,  0.23012468,  0.64707184,  0.88190687],
        [ 0.17242372,  0.32456216,  0.739125  ,  0.8860302 ],
        [-0.17786482,  0.19046786,  0.7641169 ,  0.8094367 ],
        [ 0.01500325,  0.26388648,  0.38661468,  0.90596604],
        [-0.06196262,  0.07210046,  0.54124856,  0.93361264],
        [ 0.39728656,  0.1060507 ,  1.13721   ,  0.8692758 ],
        [ 0.08947109,  0.3016826 ,  0.49939537,  0.82053614],
        [ 0.03304015,  0.10882419,  0.2325475 ,  0.58479095],
        [ 0.10821638,  0.14258239,  0.72936976,  0.6595726 ]]],
      dtype=float32)

In [49]:
classes

array([[1., 1., 1., 1., 0., 1., 1., 1., 0., 0.]], dtype=float32)

In [50]:
scores

array([[0.8504503 , 0.07880032, 0.03580749, 0.02587546, 0.02301262,
        0.02105257, 0.02009058, 0.01984097, 0.01962679, 0.01928686]],
      dtype=float32)

In [51]:
number_obj

array([10.], dtype=float32)